Для начала нужно выгрузить данные с SQL-сервера. Для этого я написал функцию, которая принимает на вход название таблицы и возвращает ее в виде pandas.DataFrame.

In [11]:
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)


In [12]:
import pandas as pd
from sqlalchemy import create_engine


query = "SELECT * FROM user_data"

for chunk in pd.read_sql(query, engine, chunksize=10000):
    # Обработка данных и обучение модели на текущем блоке данных
    pass
